In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
diabetes_data = pd.read_csv('C:\IDE\SF\PY-14_Очистка данных\Data/diabetes_data.csv')
diabetes_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


In [25]:
# 8.1

# Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных. 
# Сколько записей осталось в данных?

dupl_columns = list(diabetes_data.columns)

mask = diabetes_data.duplicated(subset=dupl_columns)
diabetes_duplicates = diabetes_data[mask]
print(f'Число найденных дубликатов: {diabetes_duplicates.shape[0]}')

diabetes_dedupped = diabetes_data.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {diabetes_dedupped.shape[0]}')

Число найденных дубликатов: 10
Результирующее число записей: 768


In [26]:
# 8.2

# Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: 
# удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны. В ответ запишите имена признаков, 
# которые вы нашли (без кавычек).

#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes_dedupped.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes_dedupped[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes_dedupped[col].nunique() / diabetes_dedupped[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

information_diabetes_data = diabetes_dedupped.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {information_diabetes_data.shape[1]}')

Gender: 100.0% одинаковых значений
Результирующее число признаков: 9


In [27]:
# 8.3

# Попробуйте найти пропуски в данных с помощью метода isnull().

# Спойлер: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. В таблице пропуски в столбцах Glucose, BloodPressure, 
# SkinThickness, Insulin и BMI обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!

# Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять 
# из библиотеки numpy: np.nan.

# Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

cols_null_percent = information_diabetes_data.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)

def nan_function(x):
    return np.nan if x == 0 else x
information_diabetes_data["Glucose"] = information_diabetes_data["Glucose"].apply(nan_function)
information_diabetes_data["BloodPressure"] = information_diabetes_data["BloodPressure"].apply(nan_function)
information_diabetes_data["SkinThickness"] = information_diabetes_data["SkinThickness"].apply(nan_function)
information_diabetes_data["Insulin"] = information_diabetes_data["Insulin"].apply(nan_function)
information_diabetes_data["BMI"] = information_diabetes_data["BMI"].apply(nan_function)
information_diabetes_data.isnull().mean().round(2).sort_values(ascending=False)

Series([], dtype: float64)

Insulin                     0.49
SkinThickness               0.30
BloodPressure               0.05
Glucose                     0.01
BMI                         0.01
Pregnancies                 0.00
DiabetesPedigreeFunction    0.00
Age                         0.00
Outcome                     0.00
dtype: float64

In [28]:
# 8.4

# Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных 
# неинформативных признаков в задании 8.2)?

diabetes = information_diabetes_data.copy()
thresh = diabetes.shape[0]*0.7
diabetes = diabetes.dropna(thresh=thresh, axis = 1)
diabetes.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'BMI',
       'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [29]:
# 8.5

# Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

diabetes = diabetes.dropna(thresh=6, axis = 0)
diabetes.shape[0]

761

In [30]:
# 8.6

# В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.

values = {
    'Pregnancies': diabetes['Pregnancies'].median(),
    'Glucose': diabetes['Glucose'].median(),
    'BloodPressure': diabetes['BloodPressure'].median(),
    'SkinThickness': diabetes['SkinThickness'].median(),
    'BMI': diabetes['BMI'].median(),
    'DiabetesPedigreeFunction': diabetes['DiabetesPedigreeFunction'].median(),
    'Age': diabetes['Age'].median(),
    'Outcome': diabetes['Outcome'].median(),
}
diabetes = diabetes.fillna(values)
#среднее значение в столбце SkinThickness
round(diabetes['SkinThickness'].mean(), 1)

29.1

In [31]:
# 8.7

# Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

# Применим эту функцию к таблице diabetes и признаку SkinThickness, а также выведем размерности результатов:

outliers, cleaned = outliers_iqr(diabetes, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


In [34]:
# 8.8

# Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

def outliers_z_score_mod(data, feature, left=3, right=3, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_z_score_mod(diabetes, 'SkinThickness')
print(outliers.shape[0])

4


In [47]:
# 8.9

# На приведённой гистограмме показано распределение признака DiabetesPedigreeFunction. Такой вид распределения очень похож на логнормальный, 
# и он заставляет задуматься о логарифмировании признака. Найдите сначала число выбросов в признаке DiabetesPedigreeFunction с помощью 
# классического метода межквартильного размаха.

def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

# Применим эту функцию к таблице diabetes и признаку DiabetesPedigreeFunction, а также выведем размерности результатов:

outliers, cleaned = outliers_iqr(diabetes, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

# Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!). 
# Какова разница между двумя этими числами (вычтите из первого второе)?

def outliers_iqr_log(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers_log = data[(x < lower_bound) | (x > upper_bound)]
    cleaned_log = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers_log, cleaned_log
# Применим эту функцию к таблице diabetes и признаку DiabetesPedigreeFunction, а также выведем размерности результатов:

outliers_log, cleaned_log = outliers_iqr_log(diabetes, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов по методу Тьюки в логарифмическом масштабе: {outliers_log.shape[0]}')
print(f'Результирующее число записей: {cleaned_log.shape[0]}')

print(f'Разница: {outliers.shape[0] - outliers_log.shape[0]}')



Число выбросов по методу Тьюки: 29
Результирующее число записей: 732
Число выбросов по методу Тьюки в логарифмическом масштабе: 0
Результирующее число записей: 761
Разница: 29
